## Celda 1: Importaciones y Configuración Inicial del Logging


In [ ]:
import pandas as pd
import json
import os
import subprocess
import logging
from datetime import datetime

# --- Clase para Logger Estilizado ---
class StyledLogger:
    def __init__(self, logger_name='StyledLogger', log_file_path='agent.log', level=logging.INFO):
        self.logger = logging.getLogger(logger_name)
        self.logger.setLevel(level)
        self.logger.propagate = False # Evitar que los logs se dupliquen si ya hay un root logger configurado

        # Limpiar handlers existentes para evitar duplicados en re-ejecuciones de celda
        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        # Formatter
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

        # File Handler
        fh = logging.FileHandler(log_file_path)
        fh.setFormatter(formatter)
        self.logger.addHandler(fh)

        # Stream Handler (consola del notebook)
        sh = logging.StreamHandler()
        sh.setFormatter(formatter)
        self.logger.addHandler(sh)

        # Estilos ASCII
        self.HEADER_ART = """
        ███████╗██╗███████╗ ██████╗ ███╗   ███╗███████╗██████╗ ███████╗██████╗ ███████╗██████╗ 
        ██╔════╝██║██╔════╝██╔════╝ ████╗ ████║██╔════╝██╔══██╗██╔════╝██╔══██╗██╔════╝██╔══██╗
        ███████╗██║███████╗██║  ███╗██╔████╔██║█████╗  ██████╔╝█████╗  ██████╔╝███████╗██████╔╝
        ╚════██║██║╚════██║██║   ██║██║╚██╔╝██║██╔══╝  ██╔══██╗██╔══╝  ██╔══██╗╚════██║██╔══██╗
        ███████║██║███████║╚██████╔╝██║ ╚═╝ ██║███████╗██║  ██║███████╗██║  ██║███████║██║  ██║
        ╚══════╝╚═╝╚══════╝ ╚═════╝ ╚═╝     ╚═╝╚══════╝╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝
        --- Google Maps Scraper Agent (Avalian Project) ---
        """
        self.SECTION_SEPARATOR = "=" * 80
        self.SUB_SECTION_SEPARATOR = "-" * 60
        self.SUCCESS_ART = "[ ✓ ]"
        self.ERROR_ART = "[ X ]"
        self.WARNING_ART = "[ ! ]"
        self.INFO_ART = "[ i ]"

    def _log(self, level, message, art=""):
        self.logger.log(level, f"{art} {message}".strip())

    def header(self):
        print(self.HEADER_ART) # Imprimir directamente para el arte ASCII grande
        self.logger.info("Agent GOSOM MVP Initialized.") # Y un log formal

    def section(self, title):
        self.logger.info(f"\n{self.SECTION_SEPARATOR}\n{title.upper()}\n{self.SECTION_SEPARATOR}")

    def subsection(self, title):
        self.logger.info(f"\n{self.SUB_SECTION_SEPARATOR}\n{title}\n{self.SUB_SECTION_SEPARATOR}")

    def info(self, message):
        self._log(logging.INFO, message, self.INFO_ART)

    def success(self, message):
        self._log(logging.INFO, message, self.SUCCESS_ART)

    def warning(self, message):
        self._log(logging.WARNING, message, self.WARNING_ART)

    def error(self, message, exc_info=False):
        self._log(logging.ERROR, message, self.ERROR_ART)
        if exc_info:
            self.logger.exception("Exception details:") # Esto capturará el traceback si se llama desde un except

    def critical(self, message):
        self._log(logging.CRITICAL, message, f"{self.ERROR_ART} [CRITICAL]")

    def debug(self, message): # Añadido para completitud
        self._log(logging.DEBUG, message, "[ D ]")


# --- Cargar Parámetros de Configuración y Inicializar Logger Estilizado ---
config_params = {}
log_file_path_global = 'agent_gmaps_mvp.log' # Fallback
try:
    # La ruta a config/parameters_default.json es relativa al notebook
    # Notebook está en: 0_AGENTE_GOSOM/notebooks/
    # Config está en: 0_AGENTE_GOSOM/config/
    # Entonces, desde el notebook, la ruta es '../config/parameters_default.json'
    config_file_relative_path = '../config/parameters_default.json'
    
    # Obtenemos la ruta absoluta del notebook para construir la ruta absoluta a config
    notebook_dir = os.path.dirname(os.path.abspath("__file__" if "__file__" in locals() else os.getcwd())) # Maneja ejecución en script y notebook
    config_file_path = os.path.join(notebook_dir, config_file_relative_path)

    with open(config_file_path, 'r') as f:
        config_params = json.load(f)
    
    # Definir la carpeta de logs relativa a la raíz del proyecto (un nivel arriba de 'notebooks')
    project_root_dir = os.path.dirname(notebook_dir)
    LOGS_FOLDER = os.path.join(project_root_dir, 'data', 'logs')
    LOG_FILENAME = config_params.get('log_filename', 'agent_gmaps_mvp.log')
    
    os.makedirs(LOGS_FOLDER, exist_ok=True)
    log_file_path_global = os.path.join(LOGS_FOLDER, LOG_FILENAME)

    # Inicializar nuestro logger estilizado
    logger = StyledLogger(log_file_path=log_file_path_global, level=logging.INFO) # Cambiar a logging.DEBUG para más detalle
    logger.header()

except FileNotFoundError:
    print(f"ERROR CRÍTICO: Archivo de configuración '{config_file_path}' no encontrado. Saliendo.")
    # En un script real, aquí harías exit(). En un notebook, podemos intentar continuar con defaults o parar.
    logger = StyledLogger() # Logger con defaults si falla la carga
    logger.critical(f"Archivo de parámetros '{config_file_path}' no encontrado. Usando defaults para logger.")
except Exception as e:
    print(f"ERROR CRÍTICO al cargar configuración o inicializar logger: {e}")
    logger = StyledLogger()
    logger.critical(f"Error al cargar configuración o inicializar logger: {e}", exc_info=True)


# --- Constantes y Parámetros Globales (leídos del JSON) ---
# Usar logger.info para mostrar que se cargaron los parámetros
logger.section("Cargando Parámetros Globales")

LANGUAGE = config_params.get('language', 'es')
logger.info(f"Lenguaje para scraping: {LANGUAGE}")

DEFAULT_DEPTH = config_params.get('default_depth', 2)
logger.info(f"Profundidad de búsqueda por defecto: {DEFAULT_DEPTH}")

RESULTS_FILENAME_PREFIX = config_params.get('results_filename_prefix', 'gmaps_data_')
logger.info(f"Prefijo para archivos de resultados: {RESULTS_FILENAME_PREFIX}")

# Rutas a carpetas de datos (relativas a la raíz del proyecto)
project_root_dir = os.path.dirname(os.path.dirname(os.path.abspath("__file__" if "__file__" in locals() else os.getcwd())))

RAW_CSV_FOLDER_NAME = config_params.get('output_csv_folder_raw_name', 'raw') # Nombre de la carpeta
RAW_CSV_FOLDER = os.path.join(project_root_dir, 'data', RAW_CSV_FOLDER_NAME)
logger.info(f"Carpeta para CSVs crudos: {RAW_CSV_FOLDER}")

PROCESSED_CSV_FOLDER_NAME = config_params.get('output_csv_folder_processed_name', 'processed') # Nombre de la carpeta
PROCESSED_CSV_FOLDER = os.path.join(project_root_dir, 'data', PROCESSED_CSV_FOLDER_NAME)
logger.info(f"Carpeta para CSVs procesados: {PROCESSED_CSV_FOLDER}")

GMAPS_COORDINATES = config_params.get('gmaps_coordinates', {})
if GMAPS_COORDINATES:
    logger.info(f"Coordenadas cargadas para {len(GMAPS_COORDINATES)} ciudades.")
else:
    logger.warning("No se cargaron coordenadas de ciudades desde la configuración.")

# Crear carpetas de datos si no existen
try:
    os.makedirs(RAW_CSV_FOLDER, exist_ok=True)
    os.makedirs(PROCESSED_CSV_FOLDER, exist_ok=True)
    logger.success("Carpetas de datos RAW y PROCESSED verificadas/creadas.")
except Exception as e:
    logger.error(f"No se pudieron crear las carpetas de datos: {e}", exc_info=True)

: 

## Celda 2 (Revisada): Funciones para Leer Archivos de Keywords


In [ ]:
# Celda 2: Funciones para Leer Archivos de Keywords

logger.section("Definiendo Funciones de Utilidad")
logger.subsection("Función para Cargar Keywords")

def load_keywords_from_csv(city_name_key):
    """
    Carga keywords desde un archivo CSV específico para una clave de ciudad.
    La clave de ciudad se usa para encontrar el archivo en ../config/keywords_<city_name_key>.csv
    """
    # La ruta a config/ es relativa al notebook
    # Notebook está en: 0_AGENTE_GOSOM/notebooks/
    # Config está en: 0_AGENTE_GOSOM/config/
    # Entonces, desde el notebook, la ruta es '../config/keywords_<city_name_key>.csv'
    
    notebook_dir = os.path.dirname(os.path.abspath("__file__" if "__file__" in locals() else os.getcwd()))
    config_dir = os.path.join(notebook_dir, '../config')
    filepath = os.path.join(config_dir, f'keywords_{city_name_key.lower()}.csv')
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            # Filtra líneas vacías y elimina espacios en blanco al principio/final
            keywords = [line.strip() for line in f if line.strip()]
        if keywords:
            logger.success(f"Keywords cargadas para '{city_name_key}' desde '{filepath}': {len(keywords)} keywords.")
        else:
            logger.warning(f"No se encontraron keywords válidas en '{filepath}' para '{city_name_key}'.")
        return keywords
    except FileNotFoundError:
        logger.error(f"Archivo de keywords no encontrado para '{city_name_key}' en '{filepath}'.")
        return []
    except Exception as e:
        logger.error(f"Error al cargar keywords para '{city_name_key}': {e}", exc_info=True)
        return []

# --- Pruebas de la función load_keywords_from_csv (Opcional) ---
logger.subsection("Probando Carga de Keywords")
test_city = 'neuquen' # Asegúrate de tener 'keywords_neuquen.csv' en ../config/
keywords_neuquen_test = load_keywords_from_csv(test_city)

if keywords_neuquen_test:
    logger.info(f"Ejemplo de keywords para {test_city.capitalize()}: {keywords_neuquen_test[:3]}...") # Muestra las primeras 3
else:
    logger.warning(f"No se pudieron cargar keywords para la prueba de {test_city.capitalize()}.")

test_city_no_existe = 'ciudad_inventada'
keywords_no_existe_test = load_keywords_from_csv(test_city_no_existe)
# (Se espera un error en el log si el archivo no existe, y la función devuelve [])